In [ ]:
# Required Packages
import requests
import os 
from PIL import Image
from IPython.display import IFrame
!pip3 install requests beautifulsoup4 --user
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import os

In [ ]:
# Glycemix index stated and less than is listed
gi_index = "120"

# Initial DataFrame Columns
food_data = pd.DataFrame(  columns = list([ "Name", "GI", "Serve (g)", "Carb Per Serve", "GL" ]) )

# Finds max number of pages
page_first = "1"
url = "http://www.glycemicindex.com/foodSearch.php?ak=list&food_name_search_type=cn&food_name=&\
gi_search_type=lte&gi={}\
&gl_search_type=lte&gl=&country=&product_category=&\
lop=AND&orderBy=GIG&find=Find+Records&page={}".format(gi_index,page_first)
html_content =requests.get(url).text
soup = BeautifulSoup(html_content, "lxml")
table = soup.find( 'table' )
table = table.find_all('p')
html_row = str(table).split('&amp;ak=detail">')[0]
max_page_num = int( re.search( "page {} of .*  -".format(page_first), html_row ).group(0).split("of")[1][1:-3] )

for page_num in range ( 1, max_page_num +1 ):
    
    # Formats url for page number
    url = "http://www.glycemicindex.com/foodSearch.php?ak=list&food_name_search_type=cn&food_name=&\
    gi_search_type=lte&gi={}\
    &gl_search_type=lte&gl=&country=&product_category=&\
    lop=AND&orderBy=GIG&find=Find+Records&page={}".format(gi_index,page_num)
    
    # Extracts table html
    html_content =requests.get(url).text
    soup = BeautifulSoup(html_content, "lxml")
    table = soup.find( 'table' )
    table = table.find_all('p')
    
    # Extracts nutrition data per page
    if re.search( "Results page '{}' not found".format(str(page_num)), str(table) ) != None:
        page_num = -1
        print("Extraction complete.")
    else:           
        for i in range ( 1, len(str(table).split('&amp;ak=detail">')) ):
            html_row = str(table).split('&amp;ak=detail">')[i]
            # print( str(table).split('&amp;ak=detail">')[i] + "\n" )
            food_name = html_row.split("</a>")[0]
            food_values = []  
            for ii in range( 1, 5 ):
                # food_values_name = [ "GI", "Serve (g)", "Carb Per Serve", "GL" ]
                food_values.append(html_row.split("</p>")[ii].split('<p class="body">')[ \
                len(html_row.split("</p>")[ii].split('<p class="body">')) -1 ] )
                page_num = page_num + 1
                
            food_data.loc[ len(food_data) ]  = ( list([food_name]) + food_values )
                
        if page_num == max_page_num:
            print("Extraction complete.")


# print( food_data )
food_data.to_excel( os.path.expanduser("~/Desktop/GI_data.xlsx") )
